In [1]:
import math
import torch
from torch import nn
from d2l import torch as d2l

In [3]:
#@save
class MultiHeadAttention(nn.Module):
    """多头注意力"""
    def __init__(self, query_size, key_size, value_size, num_hiddens,
                 num_heads, dropout, bias=False, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.attention = d2l.DotProductAttention(dropout=dropout)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=bias)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=bias)
        self.W_v = nn.Linear(value_size, num_hiddens, bias=bias)
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias)
        
    def forward(self, queries, keys, values, valid_lens):
        
        # queries 的形状：(bs, num_query, num_hiddens)
        # keys 的形状：(bs, num_k-v, num_hiddens)
        # values 的形状：(bs, num_k-v, num_hiddens)
        # after trans, the shapes will be (bs*num_heads, num_query/num_k-v, num_hiddens/num_heads)
        queries = transpose_qkv(self. (queries), self.num_heads)
        keys = transpose_qkv(self.W_k(keys), self.num_heads)
        values = transpose_qkv(self.W_v(values), self.num_heads)

        # valid_lens's shape: (bs, ) or (bs, num_query);         
        if valid_lens is not None:
            # 在轴 0， 将第一项（标量或矢量）复制 num_heads 次；然后如此复制第二项，诸如此类
            valid_lens = torch.repeat_interleave(valid_lens, repeats=self.num_heads, dim=0)
    
        # output's shape: (bs*num_heads, num_query, num_hiddens/num_heads)
        output = self.attention(queries, keys, values, valid_lens)
        
        # output_concat's shape: (bs, num_query, num_hiddens)
        output_concat = transpose_output(output, self.num_heads)
        return self.W_o(output_concat)

In [11]:
#@save
def transpose_qkv(X, num_heads):
    """
    为了多注意力头的并行计算而变换形状
    """
    
    # 输入X的形状:(batch_size, 查询或者“键-值”对的个数, num_hiddens)
    # 输出X的形状:(batch_size, 查询或者“键-值”对的个数, num_heads, num_hiddens/num_heads)
    X = X.reshape(X.shape[0], X.shape[1], num_heads, -1)
    
    # 输出X的形状:(batch_size, num_heads, 查询或者“键-值”对的个数, num_hiddens/num_heads)
    X = X.permute(0, 2, 1, 3)
    
    # 最终输出的形状:(batch_size*num_heads, 查询或者“键－值”对的个数, num_hiddens/num_heads)
    return X.reshape(-1, X.shape[2], X.shape[3])
    

In [8]:
#@save
def transpose_output(X, num_heads):
    """逆转transpose_qkv函数的操作"""
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)

下面使用键和值相同的小例子来测试我们编写的MultiHeadAttention类。 多头注意力输出的形状是（batch_size，num_queries，num_hiddens）。

In [6]:
num_hiddens, num_heads = 100, 5
attention = MultiHeadAttention(num_hiddens, num_hiddens, num_hiddens,
                               num_hiddens, num_heads, 0.5)
attention.eval()

MultiHeadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)

In [12]:
batch_size, num_queries = 2, 4
num_kvpairs, valid_lens =  6, torch.tensor([3, 2])
X = torch.ones((batch_size, num_queries, num_hiddens))
Y = torch.ones((batch_size, num_kvpairs, num_hiddens))
attention(X, Y, Y, valid_lens).shape

torch.Size([2, 4, 100])